<a href="https://colab.research.google.com/github/heghiw/lakovaci-linka/blob/main/fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pulp
import pandas as pd


import pandas as pd
import numpy as np
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.7 MB/s eta 0:00:00


In [8]:
# Remove the existing directory
!rm -rf lakovaci-linka

# Clone the repository again
!git clone https://github.com/heghiw/lakovaci-linka.git

# Navigate to the directory
%cd lakovaci-linka


Cloning into 'lakovaci-linka'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 130 (delta 60), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (130/130), 9.61 MiB | 10.54 MiB/s, done.
Resolving deltas: 100% (60/60), done.
/content/lakovaci-linka/lakovaci-linka


data prep

In [9]:
# Define the path to the Excel file
file_path = '/content/lakovaci-linka/data1.xlsx'

# Read the sheet 'linka' into a DataFrame
linka_df = pd.read_excel(file_path, sheet_name='linka')

# Read the sheet 'recept' into a DataFrame
recept_df = pd.read_excel(file_path, sheet_name='recept')

print("Linka:")
print(linka_df)
print("\nRecepty:")
print(recept_df)

Linka:
             typ vany  id_vany   pozice_x_rel  pozice_x_cum
0               vstup         0           0.0           0.0
1        Teplý oplach         1        2300.0        2300.0
2             Postřik         2        1900.0        4200.0
3    Ponor odm železo         3        1900.0        6100.0
4    Ponor odm pozink         4        1800.0        7900.0
5            oplach 1         5        1800.0        9700.0
6            oplach 2         6        1800.0       11500.0
7       Moření železo         7        1800.0       13300.0
8       Moření pozink         8        1800.0       15100.0
9      Oplach moř žel         9        1800.0       16900.0
10  Oplach moř pozink        10        1800.0       18700.0
11    oplach společný        11        1800.0       20500.0
12           aktivace        12        1800.0       22300.0
13             fosfát        13        1800.0       24100.0
14             oplach        14        1800.0       25900.0
15            Oplach         21  

manipulator and vozik characteristics

In [6]:
manipulator_characteristics = {
        "okap_point": 2000,
        "highest_point": 2750,
        "deceleration_point": 500,
        "speed_after_ponoreni": 8,
        "speed_going_up": 15,
        "speed_going_down": 12,
        "speed_left_right": 35
    }

In [10]:
linka_df.columns = linka_df.columns.str.strip()
recept_df.columns = recept_df.columns.str.strip()
print(linka_df.columns)
print(recept_df.columns)

Index(['typ vany', 'id_vany', 'pozice_x_rel', 'pozice_x_cum'], dtype='object')
Index(['tech', 'id_vany', 'poradi_operace', 'cas_min', 'cas_max', 'cas_opt',
       'okap', 'okap_cas'],
      dtype='object')


### Transition Matrix Definition

We define the transition time matrix \( T \in \mathbb{R}^{n \times n} \) as:

\[
T_{i,j} =
\begin{cases}
\text{dwell}_j, & \text{if } i = j \quad \text{(product dwell)} \\\\
\text{move}(i, j) + \text{lower} + \text{raise}(j), & \text{if } i < j \quad \text{(product forward move)} \\\\
\text{move}(i, j), & \text{if } i > j \quad \text{(empty/utility move)}
\end{cases}
\]

---

### Vertical Raise Logic

\[
\text{raise}(j) =
\begin{cases}
\displaystyle
\frac{h_\text{bottom} - h_\text{okap}}{v_\text{up}} + t_\text{okap} + \frac{h_\text{okap} - h_\text{top}}{v_\text{up}}, & \text{if okap}_j = 1 \\\\
\displaystyle
\frac{h_\text{bottom} - h_\text{top}}{v_\text{up}}, & \text{if okap}_j = 0
\end{cases}
\]

---

### Vertical and Horizontal Movement

\[
\text{lower} = \frac{h_\text{top} - h_\text{bottom}}{v_\text{down}}, \quad
\text{move}(i, j) = \frac{|x_j - x_i|}{v_\text{horizontal}}
\]

---

### Parameters from Simulation

- \( h_\text{top} = 2750 \,\text{mm} \)
- \( h_\text{okap} = 2000 \,\text{mm} \)
- \( h_\text{bottom} = 0 \,\text{mm} \)
- \( v_\text{horizontal} = 35 \,\text{m/min} \)
- \( v_\text{down} = 12 \,\text{m/min} \)
- \( v_\text{up} = 15 \,\text{m/min} \)
- \( t_\text{okap} \): pause time at okap height (from recipe)
- \( \text{okap}_j \): indicator whether okap is required at bath \( j \)


In [15]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm
import math

# ---- CONFIG ----
manipulator_characteristics = {
    "okap_point": 2000,
    "highest_point": 2750,
    "deceleration_point": 500,
    "speed_after_ponoreni": 8,
    "speed_going_up": 15,
    "speed_going_down": 12,
    "speed_left_right": 35
}
takt_limit = 300  # seconds

# ---- HELPER FUNCTIONS ----

def get_distance(x1, x2):
    return abs(x1 - x2)

def vertical_time(down=True, okap_time=0):
    mc = manipulator_characteristics
    if down:
        distance = mc["highest_point"]
        speed = mc["speed_going_down"]
        return (distance / 1000) / (speed / 60)
    else:
        h1 = (mc["highest_point"] - mc["okap_point"]) / 1000
        h2 = mc["okap_point"] / 1000
        v_up = mc["speed_going_up"] / 60  # m/s
        return (h1 / v_up) + okap_time + (h2 / v_up)

def horizontal_time(pos1, pos2):
    dist = get_distance(pos1, pos2)
    return (dist / 1000) / (manipulator_characteristics["speed_left_right"] / 60)

def move_time(i, j, linka_df, okap_time_dict=None):
    pos_i = linka_df.loc[i, "pozice_x_cum"] if i in linka_df.index else 0
    pos_j = linka_df.loc[j, "pozice_x_cum"] if j in linka_df.index else 0
    move = horizontal_time(pos_i, pos_j)

    if i == j:
        return 0.0
    if i > j:
        return move

    down = vertical_time(down=True)
    up = vertical_time(down=False, okap_time=okap_time_dict.get(i, 0))
    return move + down + up

def sample_dwell(row):
    a, b, mu = row["cas_min"], row["cas_max"], row["cas_opt"]
    if b <= a or np.isnan(a) or np.isnan(b) or np.isnan(mu):
        return mu
    sigma = (b - a) / 6
    return truncnorm((a - mu) / sigma, (b - mu) / sigma, loc=mu, scale=sigma).rvs()

# ---- CORRIDOR ASSIGNMENT FUNCTION WITH OVERLAP ----
def assign_corridors_with_overlap(bath_ids, linka_df, num_manips, overlap=1):
    bath_pos = linka_df.set_index("id_vany")["pozice_x_cum"].to_dict()
    bath_pos[0] = -1000
    bath_pos[100] = 100000
    bath_ids = sorted(bath_ids, key=lambda b: bath_pos.get(b, float('inf')))

    n = len(bath_ids)
    base_size = math.ceil(n / num_manips)
    corridors = []

    for i in range(num_manips):
        start = max(0, i * base_size - overlap)
        end = min(n, (i + 1) * base_size + overlap)
        corridor = bath_ids[start:end]
        corridors.append(sorted(set(corridor), key=lambda b: bath_pos.get(b, float('inf'))))

    return corridors

# ---- MAIN LOGIC ----

tech = input("Enter technology (e.g., 'tech1'): ").strip()
recipe = recept_df[recept_df.tech == tech].copy()
recipe = recipe.sort_values("poradi_operace")

# Include vstup (0) and vystup (100)
recipe_baths = [0] + recipe["id_vany"].tolist() + [100]
all_baths = pd.DataFrame({'id_vany': recipe_baths}).drop_duplicates()

# Map physical positions
linka_positions = linka_df.set_index("id_vany")["pozice_x_cum"]
all_baths["pozice_x_cum"] = all_baths["id_vany"].map(linka_positions).fillna(-1)
all_baths.loc[all_baths["id_vany"] == 0, "pozice_x_cum"] = -1000
all_baths.loc[all_baths["id_vany"] == 100, "pozice_x_cum"] = 100000
unique_baths = all_baths.sort_values("pozice_x_cum")["id_vany"].tolist()

# Sample dwell times
dwell_times = {row.id_vany: sample_dwell(row) for _, row in recipe.iterrows()}
dwell_times[0] = 0
dwell_times[100] = 0

# Okap info
okap_time_dict = {row.id_vany: row.okap_cas if row.okap else 0 for _, row in recipe.iterrows()}
okap_time_dict[0] = 0
okap_time_dict[100] = 0

# Transition Matrix
matrix = pd.DataFrame(index=unique_baths, columns=unique_baths, dtype=float)
for i in unique_baths:
    for j in unique_baths:
        if i == j:
            matrix.loc[i, j] = dwell_times.get(i, 0)
        else:
            matrix.loc[i, j] = move_time(i, j, linka_df.set_index("id_vany"), okap_time_dict)

print("\n--- TRANSITION MATRIX for", tech, "---")
print(matrix.round(1).to_string())

# Manipulator count suggestion
max_manips = len(unique_baths) - 1
suggested = math.ceil(len(unique_baths) / 2)
print("\n✅ Suggested Manipulator Count:", suggested)

# Manual override
user_input = input("Enter number of manipulators (or press Enter to use suggestion): ").strip()
num_manips = suggested if not user_input else int(user_input)

# Assign overlapping corridors
corridors = assign_corridors_with_overlap(unique_baths, linka_df, num_manips, overlap=1)

# Display corridor assignments
print("\n--- MANIPULATOR CORRIDORS (With Overlap) ---")
for i, group in enumerate(corridors, 1):
    print(f"  M{i}: Baths {group}")

# Schedule product path
print("\n--- PRODUCT PATH SCHEDULE ---")
print("   Step | From → To   |  Time Used (s)")
print("--------------------------------------")
path = [0] + recipe["id_vany"].tolist() + [100]
time = 0
for i in range(1, len(path)):
    a, b = path[i-1], path[i]
    t = matrix.loc[a, b]
    print(f"{i:7} | {a:>4} → {b:<4}  | {time:6.1f} → {time + t:6.1f}")
    time += t

# Cyclic manipulator paths
print("\n--- CYCLIC SCHEDULE PER MANIPULATOR ---")
print("   Manip | From → To   |  Start → End   ")
print("----------------------------------------")

for idx, group in enumerate(corridors):
    manip = f"M{idx+1}"
    if len(group) < 2:
        continue
    t = 0
    path = group.copy()
    for i in range(1, len(path)):
        a, b = path[i-1], path[i]
        move_t = matrix.loc[a, b]
        print(f"{manip:>8} | {a:>4} → {b:<4}  | {t:6.1f} → {t + move_t:6.1f}")
        t += move_t
    back = matrix.loc[path[-1], path[0]]
    print(f"{manip:>8} | {path[-1]:>4} → {path[0]:<4}  | {t:6.1f} → {t + back:6.1f}")


Enter technology (e.g., 'tech1'): tech1

--- TRANSITION MATRIX for tech1 ---
      0      1      2      3     5     6      7     9     11     12     13    14    21    15    16     17     18     19     20     100
0     0.0   28.7   32.0   35.2  41.4  44.5   47.6  53.7  59.9   63.0   66.1  69.1  72.2  75.3  78.4   82.4   86.3   89.4   92.5   96.4
1     3.9  112.8   48.0   51.3  57.4  60.5   63.6  69.8  75.9   79.0   82.1  85.2  88.3  91.4  94.5   98.4  102.4  105.4  108.5  112.5
2     7.2    3.3  147.9   28.0  34.2  37.3   40.3  46.5  52.7   55.8   58.9  62.0  65.0  68.1  71.2   75.1   79.1   82.2   85.3   89.2
3    10.5    6.5    3.3  224.6  50.9  54.0   57.1  63.3  69.4   72.5   75.6  78.7  81.8  84.9  87.9   91.9   95.8   98.9  102.0  106.0
5    16.6   12.7    9.4    6.2  66.9  42.8   45.9  52.1  58.3   61.4   64.4  67.5  70.6  73.7  76.8   80.7   84.7   87.8   90.8   94.8
6    19.7   15.8   12.5    9.3   3.1  66.6   42.8  49.0  55.2   58.3   61.4  64.4  67.5  70.6  73.7   77.6   81.6